In [11]:
import pandas as pd
import calendar
# import ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import gzip

In [12]:
# Loading data from CSV files into pandas DataFrames

consumption_df = pd.read_csv('./consumo_energia_eletrica.csv')
states_df = pd.read_csv('./estado_regiao.csv')

with gzip.open('./populacao.csv', 'rb') as f_in:
    population_df = pd.read_csv(f_in)

del population_df['populacao_economicamente_ativa']

In [13]:
# check duplicated records
get_duplicated = consumption_df.duplicated()
print(f'total number of duplicate lines: {get_duplicated.sum() }')

total number of duplicate lines: 1017


In [14]:
# search for null fields

null_df = consumption_df.isnull()

print(null_df.sum())

ano                        0
mes                        0
sigla_uf                   0
tipo_consumo               0
numero_consumidores    12960
consumo                    0
dtype: int64


In [15]:
# Check if the total value is the sum of other values

consumption_2004_df = consumption_df[
    (consumption_df['ano'] == 2022) &
    (consumption_df['sigla_uf'] == 'PE') &
    (consumption_df['mes'] == 2)
]

total_df = consumption_2004_df[
    consumption_2004_df['tipo_consumo'] == 'Total'
]

others_df = consumption_2004_df[
    consumption_2004_df['tipo_consumo'] != 'Total'
]

# sum types
# others_sum = (
#     others_df
#     .groupby(['ano', 'mes', 'sigla_uf'], as_index=False)['consumo']
#     .sum()
# )

print(total_df)
print(others_df)

        ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
35031  2022    2       PE        Total                  NaN  1282724
        ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
35350  2022    2       PE       Cativo                  NaN   872075
35679  2022    2       PE       Outros             171780.0   255068
35995  2022    2       PE    Comercial             226681.0   248323
36325  2022    2       PE   Industrial               4821.0   321438
36652  2022    2       PE  Residencial            3482337.0   457895


In [16]:
population_df.head()

,sigla_uf,ano,populacao
0,AC,1991,417102
1,AL,1991,2512658
2,AM,1991,2102766
3,AP,1991,289035
4,BA,1991,11867336


In [17]:
# Filtra as linhas com tipo_consumo "Total"
total_consumo = consumption_df[consumption_df["tipo_consumo"] == "Total"]

# Conta quantos valores de numero_consumidores são nulos
nulos = total_consumo["numero_consumidores"].isna().sum()

print("Número de registros com numero_consumidores nulo para tipo 'Total':", nulos)

Número de registros com numero_consumidores nulo para tipo 'Total': 6480


In [22]:
# function to count tipo_consumo Cativo missing values
def count_consumpion_type_captive(df, consumption_type): 
    df = df[df["tipo_consumo"] == consumption_type]    
    return df["numero_consumidores"].isna().sum()

count_captive = count_consumpion_type_captive(consumption_df, 'Cativo')
print(f'\nTotal number of missing values for the numero_consumidores field with Cativo tipo_consumo: {count_captive}')

# removing tipo_consumo Cativo from dataframe
consumption_transformed_df = consumption_transformed_df[~((consumption_transformed_df["tipo_consumo"] == "Cativo") & (consumption_transformed_df["numero_consumidores"].isna()))]

count_captive = count_consumpion_type_captive(consumption_transformed_df, 'Total')
print(f'\nTotal number of missing values for the numero_consumidores field with Cativo tipo_consumo: {count_captive}')


Total number of missing values for the numero_consumidores field with Cativo tipo_consumo: 0

Total number of missing values for the numero_consumidores field with Cativo tipo_consumo: 6480


In [24]:
# Supondo que seu DataFrame original se chame df
# df = pd.read_csv(...) ou já existente

# 1️⃣ Separar os registros que não são Total
df_tipos = consumption_transformed_df[consumption_transformed_df["tipo_consumo"] != "Total"].copy()

# 2️⃣ Agrupar por ano, mes, sigla_uf para calcular soma de numero_consumidores e consumo
df_total_calculado = df_tipos.groupby(
    ["ano", "mes", "sigla_uf"], as_index=False
).agg({
    "numero_consumidores": "sum",
    "consumo": "sum"
})

# 3️⃣ Adicionar a coluna tipo_consumo = "Total"
df_total_calculado["tipo_consumo"] = "Total"

# 4️⃣ Reordenar colunas igual ao original (opcional)
df_total_calculado = df_total_calculado[["ano", "mes", "sigla_uf", "tipo_consumo", "numero_consumidores", "consumo"]]

# Agora temos:
# df_tipos -> com todos os tipos detalhados, sem os registros de Total
# df_total_calculado -> com o tipo Total recalculado a partir da soma dos demais tipos

# ✅ Verificando os dataframes
print("Tipos detalhados (sem Total):")
print(df_tipos.head())

print("\nTotal recalculado:")
print(df_total_calculado.head())

Tipos detalhados (sem Total):
      ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
648  2004    1       RN       Outros              40857.0    69617
649  2004    1       SP       Outros             311650.0   937538
650  2004    1       MS       Outros              56881.0    67601
651  2004    1       SC       Outros             226165.0   209380
652  2004    1       RJ       Outros              70634.0   416128

Total recalculado:
     ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
0   2004    1       AC        Total             126275.0    34839
1   2004    1       AL        Total             637016.0   285489
2   2004    1       AM        Total             543155.0   274773
3   2004    1       AP        Total             114101.0    43084
4   2004    1       BA        Total            3502266.0  1444449
5   2004    1       CE        Total            2148029.0   518714
6   2004    1       DF        Total             662860.0   311459
7   2004    1       